# MODELING

In [3]:
from tqdm import tqdm
from collections import defaultdict
import gensim
from gensim.corpora import Dictionary
from gensim.models import Phrases
from gensim.models import LdaModel
import pyLDAvis.gensim
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymystem3 import Mystem
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import seaborn as sns
%matplotlib inline

/Users/evgeny_yu/anaconda/lib/python3.6/site-packages/nltk/decorators.py:59: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  regargs, varargs, varkwargs, defaults = inspect.getargspec(func)


# Select the neighborhood and load its data

In [4]:
name = 'Akademicheskij'
district = gpd.read_file('../Data dive/dd2/{}/{}_district.geojson'.format(name,name))
neigh_posts = pd.read_csv('social_media/{}/vk.csv'.format(name))

In [5]:
neigh_posts

,Unnamed: 0,lat,lon,userId,date,text,place_id,hour,minutes,day,weekday,geometry
0,92844,55.690736,37.575364,2812735,1464748216,"Разгульный образ жизни, знаете ли #me #walk #m...",4166,18000,30,1,2,POINT (37.575363805272 55.690735940851)
1,92845,55.690736,37.575364,2812735,1464748203,"Еда не осудит. Еда не предаст, еда не скажет ""...",4166,18000,30,1,2,POINT (37.575363805272 55.690735940851)
2,92846,55.690736,37.575364,25685742,1464647680,Лена и Феррари 💃#еленапрекрасная#машина#незнаю...,4166,3600,34,30,0,POINT (37.575363805272 55.690735940851)
3,92847,55.690736,37.575364,1895976,1464480464,Не останавливаемся🍷🍷🍷 @ Кафе Шоколад Москва Ул...,4166,10800,7,28,5,POINT (37.575363805272 55.690735940851)
4,92848,55.690736,37.575364,93684102,1464273230,#выходныеудались #шоколад @ Кафе Шоколад Мос...,4166,61200,33,26,3,POINT (37.575363805272 55.690735940851)
5,92849,55.690736,37.575364,93684102,1464221426,Выходные 💃💃 #шоколад #клубшоколад #вклубешнике...,4166,10800,10,25,2,POINT (37.575363805272 55.690735940851)
6,92850,55.690736,37.575364,196913879,1464220064,#шоколадки😀💃🏻💃🏻🍫 @ Кафе Шоколад Москва Ул. 60-...,4166,7200,47,25,2,POINT (37.575363805272 55.690735940851)
7,92851,55.690736,37.575364,34889724,1464122864,🎉🎉🎉<br> @ Кафе Шоколад Москва Ул. 60-Летия Окт...,4166,82800,47,24,1,POINT (37.575363805272 55.690735940851)
8,92852,55.690736,37.575364,348625783,1464027017,🍕 #pizza #отдых @Кафе Шоколад Москва Ул. 60-Л...,4166,75600,10,23,0,POINT (37.575363805272 55.690735940851)
9,92853,55.690736,37.575364,19919645,1463877858,Пожалуй после этих выходных я сделаю паузу😂😱😝🙈...,4166,10800,44,21,5,POINT (37.575363805272 55.690735940851)


# Group posts by user

In [6]:
vk_users = pd.DataFrame({'post' : neigh_posts.groupby('userId').apply( lambda x: ' '.join(x['text']))}).reset_index()

In [7]:
vk_users

,userId,post
0,967,#музейдарвина @ Государственный Дарвиновский м...
1,1883,Это тема) @ HookahPlace Grimau
2,2187,Ара в магазине Папа Карло на Профсоюзной. @ Зо...
3,7322,Спасибо за отличную поездку)
4,8006,Молодежка-3! Соскучились?))<br>Поехали!!! 🎬🎥 #...
5,8625,Ланчкофебрейк ^^
6,11949,Made with @instaweatherpro Free App! #instawea...
7,12582,Ко мне пришёл позитивный Дед Мороз со Снегуроч...
8,14538,И абсолютно FABULOUS LAS VEGAS!!! 😍 #makbeerop...
9,15250,Sooo white as snow ❄⛄❄<br>#снег #rhcp #снежное...


# How many users are there?

In [8]:
extra_words = ['http','br','id','com','www', 'instagram', 'vsco', 'https', 'instasize','repost',
              'whatsapp', 'вотсап', 'repostapp','маникюр', 'бровь', 'ресница', 'губа', 'instacollage', 'опубликовывать',
                'фото', 'москва', 'moscow']
def process_docs(docs):
    """
    Function to process texts. Following are the steps we take:
    
    1. Text tokenization.
    2. Removing numbers 
    3. Stopword and short words Removal.
    4. Lemmatization and filter words by their length.
    
    Args:
    ----------
    texts: Tokenized texts.
    
    Returns:
    -------
    texts: Pre-processed tokenized texts.
    """
    m = Mystem()
    # Split the documents into tokens.
    tokenizer = RegexpTokenizer(r'\w+')
    stops = stopwords.words('russian') + stopwords.words('english') + extra_words
    
    for idx in tqdm(range(len(docs))):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not any(c.isdigit() for c in token) and ('id' not in token or 'club' not in token or 'ru' not in token)] for doc in tqdm(docs)]
    #Lemmatize words
    docs = [[m.lemmatize(token)[0] for token in doc ] for doc in tqdm(docs)]
    #Remove stopwords
    docs = [[token for token in doc if token not in stops] for doc in tqdm(docs)]
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in tqdm(docs)]
    return docs
    
def get_corpus(docs):
    
    """Add bigrams to docs and create corpus and dictionary for training
    
    Args:
        docs: list of tokenized and cleaned texts;
    Returns:
        corpus: list of lists of tuples, where first element of tuple is a word id
        and the second is the count of that word in the whole corpus
        dictionary: gensim.corpora.dictionary.Dictionary 
  
    """
    
    frequency = defaultdict(int)
    for text in tqdm(docs):
        for token in text:
            frequency[token] += 1

    texts = [[token for token in text if frequency[token] > 3] for text in tqdm(docs)]

    #Take the bigram, if token is a bigram, add to document.
    bigram = Phrases(texts, min_count = 20)
    for idx in tqdm(range(len(texts))):
        for token in bigram[texts[idx]]:
            if '_' in token:
                texts[idx].append(token)
    
    # Create a dictionary representation of the documents.
    dictionary = Dictionary(texts)
    # Filter out words that occur less than 20 documents, or more than 50% of the documents.
    #dictionary.filter_extremes(no_below=20, no_above=0.5)
    corpus = [dictionary.doc2bow(doc) for doc in tqdm(texts)]
    
    print('Number of unique tokens: {}'.format(len(dictionary)))
    print('Number of documents: {}'.format(len(corpus)))
    
    return corpus, dictionary

# Process texts to be ready for modeling

In [9]:
texts = vk_users['post'].copy()

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evgeny_yu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
docs = process_docs(texts.values)

100%|██████████| 9133/9133 [00:00<00:00, 231589.83it/s]


In [12]:
corpus, dictionary = get_corpus(docs)

  0%|          | 0/9133 [00:00<?, ?it/s]/Users/evgeny_yu/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
100%|██████████| 9133/9133 [00:00<00:00, 47384.33it/s]


Number of unique tokens: 5679
Number of documents: 9133


In [13]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


#The training model - we use online LDA model which allows to update the model 
#and the following parameters should be defined
num_topics = 10 # number of topics
chunksize = 1000 
passes = 10
iterations = 400
eval_every = 10  #evaluate model perplexity.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha=0.001, update_every = 1, \
                       num_topics=num_topics,\
                       eval_every=eval_every, passes = passes)

2017-05-12 10:24:55,912 : INFO : using symmetric eta at 0.00017608733932030288
2017-05-12 10:24:55,915 : INFO : using serial LDA version on this node
2017-05-12 10:24:56,302 : INFO : running online LDA training, 10 topics, 10 passes over the supplied corpus of 9133 documents, updating model once every 1000 documents, evaluating perplexity every 9133 documents, iterating 50x with a convergence threshold of 0.001000
2017-05-12 10:24:56,303 : INFO : PROGRESS: pass 0, at document #1000/9133
2017-05-12 10:24:57,283 : INFO : merging changes from 1000 documents into a model of 9133 documents
2017-05-12 10:24:57,343 : INFO : topic #4 (0.001): 0.028*"академический" + 0.019*"академический_район" + 0.019*"район" + 0.008*"улица" + 0.007*"profsoyuznaya" + 0.007*"музей" + 0.007*"очень" + 0.006*"спасибо" + 0.006*"утро" + 0.006*"дарвиновский"
2017-05-12 10:24:57,345 : INFO : topic #2 (0.001): 0.020*"музей" + 0.014*"государственный" + 0.014*"друг" + 0.013*"дарвиновский" + 0.011*"государственный_дарвино

CPU times: user 44.4 s, sys: 883 ms, total: 45.3 s
Wall time: 46.1 s


In [ ]:
# import nltk
# nltk.download()

In [14]:
data = pyLDAvis.gensim.prepare(model, corpus, dictionary); # visualize lda topics
pyLDAvis.display(data)

# Assign topics to users

In [17]:
docTopicProbMat = model[corpus]
lda_users = vk_users.copy()
lda_users['topics'] = docTopicProbMat
vk_users['topic'] = lda_users['topics'].apply(lambda x :x[0][0])

# Assign topics to their posts

In [18]:
neigh_posts['topic'] = neigh_posts['userId'].apply(lambda userId: vk_users.loc[vk_users['userId']==userId,'topic'].item())

# Plot the histogram of topic distribution per posts and per users

# Mapping

In [19]:
center_lat = list(district.centroid[0].coords)[0][1]
center_lon = list(district.centroid[0].coords)[0][0]

In [23]:
map_places = folium.Map([center_lat, center_lon], tiles='Stamen Toner', zoom_start=14,control_scale=True)

#Define style for geojson objects
style_function = lambda feature: dict(fillColor='#AECCAE',
                                      color='#AECCAE',
                                      weight=1,
                                      opacity=0.3)

# Adding Houses
houses = gpd.read_file('../Data dive/dd2/{}/{}_chruchevki.geogson'.format(name, name))
points = folium.features.GeoJson(houses,name='Khurshevki houses')
map_places.add_child(points,name='Khurshevki houses')

#Adding district
polygon = folium.features.GeoJson(district, style_function=style_function,name='district boundary')
map_places.add_child(polygon,name='district boundary')

colormap_dict = {0.0: 'pink', 0.3: 'blue', 0.5: 'green',  1.0: 'red'}

#Adding topics heatmaps
for topic_id in [0, 1, 2, 5]:
    topic_coords = list(zip(neigh_posts[neigh_posts.topic == topic_id].lat, neigh_posts[neigh_posts.topic == topic_id].lon))

    HeatMap(topic_coords,
            name='Topic: {}'.format(topic_id),
            radius=10, 
            min_opacity=0.8,
            gradient={0.0: 'pink', 0.3: 'blue', 0.5: 'green',  1.0: 'red'}).add_to(map_places)


    colormap = folium.LinearColormap(colors = colormap_dict.values())
    colormap.caption = 'Topic: {}'.format(topic_id)

#Switch between layers
folium.LayerControl().add_to(map_places)
map_places